In [1]:
# Cell 1: 随机生成示例 anomaly maps 与整数缺陷掩码，并写入文件
import numpy as np, json, os

# 可调参数
N = 5          # 图片数量
H, W = 256, 256
RADIUS_RANGE = (10, 35)   # 缺陷半径范围
PIXEL_VALUES = [255, 254, 253]  # 与 defects_config.json 中的 pixel_value 对应
OUTPUT_DIR = os.getcwd()  # 当前 notebook 目录 (toy_exp)

rng = np.random.default_rng(42)

amaps = np.zeros((N, H, W), dtype=np.float32)
masks = np.zeros((N, H, W), dtype=np.uint16)

for i in range(N):
    num_defects = rng.integers(1, len(PIXEL_VALUES) + 1)  # 每张图随机几种缺陷
    chosen_defects = rng.choice(PIXEL_VALUES, size=num_defects, replace=False)

    for pv in chosen_defects:
        cx = rng.integers(RADIUS_RANGE[1], H - RADIUS_RANGE[1])
        cy = rng.integers(RADIUS_RANGE[1], W - RADIUS_RANGE[1])
        radius = rng.integers(*RADIUS_RANGE)
        yy, xx = np.ogrid[:H, :W]
        circle = (yy - cx) ** 2 + (xx - cy) ** 2 <= radius ** 2
        masks[i][circle] = pv
        # 生成高斯型异常分数（越靠近中心越高）
        dist = (yy - cx) ** 2 + (xx - cy) ** 2
        blob = np.exp(-dist / (2 * (radius * 0.6) ** 2))
        amaps[i] += blob.astype(np.float32)

    # 加少量噪声
    amaps[i] += 0.05 * rng.standard_normal((H, W)).astype(np.float32)
    # 归一化到 [0,1]
    mmin, mmax = amaps[i].min(), amaps[i].max()
    if mmax > mmin:
        amaps[i] = (amaps[i] - mmin) / (mmax - mmin)

# 保存到文件，供后续 cell 使用
np.save(os.path.join(OUTPUT_DIR, 'anomaly_maps.npy'), amaps)
np.save(os.path.join(OUTPUT_DIR, 'gt_integer_masks.npy'), masks)

# 构造与上述 pixel values 对应的 defects_config.json
# saturation_threshold=1.0 且 relative_saturation=True -> 饱和面积 = 缺陷面积 * 1.0
entries = [
    {
        "defect_name": f"defect_{pv}",
        "pixel_value": int(pv),
        "saturation_threshold": 1.0,
        "relative_saturation": True
    } for pv in PIXEL_VALUES
]
with open(os.path.join(OUTPUT_DIR, 'defects_config.json'), 'w') as f:
    json.dump(entries, f, indent=2)

print('Generated amaps shape:', amaps.shape)
print('Generated masks shape:', masks.shape)
print('Defects config saved to defects_config.json with pixel_values:', PIXEL_VALUES)

Generated amaps shape: (5, 256, 256)
Generated masks shape: (5, 256, 256)
Defects config saved to defects_config.json with pixel_values: [255, 254, 253]


In [6]:
import sys
import os
import numpy as np # 把 import 都放在前面

# 1. 确定项目根目录 (你的 .ipynb 在 'toy_exp/' 中, '..' 指向根目录)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 2. (关键!) 先把根目录添加到 sys.path
if project_root not in sys.path:
    # 插入到列表开头 (0)，确保它被优先搜索
    sys.path.insert(0, project_root)
    print(f"Added project root to path: {project_root}")

# 3. 现在再导入，因为根目录已在搜索路径中
# Python 会在根目录找到 'metrics.py' 文件并从中导入
from metrics import compute_auc_spro_curve_from_defects_config
result = compute_auc_spro_curve_from_defects_config(
    masks=masks,
    amaps=amaps,
    defects_config_path="defects_config.json",  # 使用生成的文件
    max_fprs=[0.01, 0.05, 0.1, 0.3, 1.0],
    num_thresholds=200,
    structuring_element_size=1,  # 保持与 LOCO 论文一致
)

print("AUC-sPRO values:", result["auc_spro"])  # 各积分上限的结果
# 还可访问曲线：
fprs = result["fpr_curve"]
spros = result["spro_curve"]
print("FPR curve length:", len(fprs), "sPRO curve length:", len(spros))
print("First 5 FPRs:", fprs[:5])
print("First 5 sPROs:", spros[:5])

Added project root to path: /home/voidsolar/project/LogicAnomaly
AUC-sPRO values: {0.01: np.float64(0.8241126504453867), 0.05: np.float64(0.9408128167198938), 0.1: np.float64(0.9679140825698219), 0.3: np.float64(0.989057872637051), 1.0: np.float64(0.9967173617911153)}
FPR curve length: 200 sPRO curve length: 200
First 5 FPRs: [0. 0. 0. 0. 0.]
First 5 sPROs: [0.32106593 0.29334891 0.26159644 0.24852522 0.27264085]
AUC-sPRO values: {0.01: np.float64(0.8241126504453867), 0.05: np.float64(0.9408128167198938), 0.1: np.float64(0.9679140825698219), 0.3: np.float64(0.989057872637051), 1.0: np.float64(0.9967173617911153)}
FPR curve length: 200 sPRO curve length: 200
First 5 FPRs: [0. 0. 0. 0. 0.]
First 5 sPROs: [0.32106593 0.29334891 0.26159644 0.24852522 0.27264085]
